In [30]:
import langchain
import groq
import chromadb
import bs4
import requests
import pdfplumber
import sentence_transformers
print("All libraries installed successfully!")

All libraries installed successfully!


In [31]:
import os
os.environ["GROQ_API_KEY"] = "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"  # Replace with your full key

# Verify API key with Groq
from groq import Groq
client = Groq()
try:
    models = client.models.list()
    print("Available models:", [model.id for model in models.data])
except Exception as e:
    print(f"API key error: {str(e)}")

Available models: ['qwen/qwen3-32b', 'meta-llama/llama-guard-4-12b', 'playai-tts', 'meta-llama/llama-prompt-guard-2-86m', 'whisper-large-v3-turbo', 'moonshotai/kimi-k2-instruct', 'meta-llama/llama-prompt-guard-2-22m', 'playai-tts-arabic', 'llama-3.1-8b-instant', 'openai/gpt-oss-120b', 'llama-3.3-70b-versatile', 'moonshotai/kimi-k2-instruct-0905', 'meta-llama/llama-4-scout-17b-16e-instruct', 'whisper-large-v3', 'meta-llama/llama-4-maverick-17b-128e-instruct', 'openai/gpt-oss-20b', 'groq/compound-mini', 'allam-2-7b', 'groq/compound']


In [32]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate

# Load the Groq LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.1
)

# Define a prompt template for summarization
prompt = ChatPromptTemplate.from_template(
    "Summarize the following paragraph in 2 sentences: {paragraph}"
)

# Chain the prompt with the LLM
chain = prompt | llm

try:
    # Test invocation
    response = chain.invoke({
        "paragraph": "Transformers are a type of neural network architecture introduced in 2017 that revolutionized NLP by using self-attention mechanisms."
    })
    print("Success! Summary:", response.content)
except Exception as e:
    print(f"Error during invocation: {str(e)}")
    print("Try fallback model: 'gemma-7b-it'. Check https://console.groq.com/docs/models for latest.")

Success! Summary: Here's a 2-sentence summary of the paragraph:

Transformers are a type of neural network architecture introduced in 2017 that have significantly impacted the field of Natural Language Processing (NLP). They achieved this by utilizing self-attention mechanisms, a key innovation that has revolutionized the way NLP models process and understand language.


In [33]:
# Cell 8: Test summarization
summarize_prompt = ChatPromptTemplate.from_template("Summarize this paragraph: {text}")
summarize_chain = summarize_prompt | llm

text = "Large Language Models (LLMs) are advanced AI systems trained on massive datasets to generate human-like text. They power applications like chatbots, translation, and content generation."
response = summarize_chain.invoke({"text": text})
print("Summary:", response.content)

Summary: Large Language Models (LLMs) are advanced AI systems trained on large datasets to generate human-like text, and are used in applications such as chatbots, translation, and content creation.


In [34]:
# Cell 5: Define Web Crawler and PDF Scraper tools
from langchain.tools import tool
import requests
import re
from bs4 import BeautifulSoup
import pdfplumber

@tool
def web_crawler(url: str) -> str:
    """Fetches plain text content from a webpage URL."""
    try:
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        text = ' '.join([p.get_text() for p in soup.find_all(['p', 'h1', 'h2', 'h3'])])
        return text.strip()[:2000]
    except Exception as e:
        return f"Error fetching URL: {str(e)}"

@tool
def research_paper_scraper(pdf_path: str) -> str:
    """Extracts abstract, introduction, and conclusion from a PDF research paper."""
    try:
        with pdfplumber.open(pdf_path) as pdf:
            text = ""
            current_section = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    lines = page_text.split('\n')
                    for line in lines:
                        line_lower = line.lower()
                        if "abstract" in line_lower:
                            current_section = "abstract"
                        elif "introduction" in line_lower:
                            current_section = "introduction"
                        elif "conclusion" in line_lower:
                            current_section = "conclusion"
                        if current_section:
                            text += line + " "
            return text.strip()[:2000]
    except Exception as e:
        return f"Error processing PDF: {str(e)}"

print("Tools defined successfully!")

Tools defined successfully!


In [35]:
# Cell 6: Test web crawler
url = "https://en.wikipedia.org/wiki/Transformer_(deep_learning_architecture)"
try:
    global web_result
    web_result = web_crawler.invoke(url)
    print("Web Crawler Output:\n" + "-"*50 + "\n" + web_result + "\n" + "-"*50)
except Exception as e:
    print(f"Error: {str(e)}")
    print("Try alternative URL: https://huggingface.co/blog/transformers")

Web Crawler Output:
--------------------------------------------------
Contents Transformer (deep learning architecture) In deep learning, the transformer is a neural network architecture based on the multi-head attention mechanism, in which text is converted to numerical representations called tokens, and each token is converted into a vector via lookup from a word embedding table.[1] At each layer, each token is then contextualized within the scope of the context window with other (unmasked) tokens via a parallel multi-head attention mechanism, allowing the signal for key tokens to be amplified and less important tokens to be diminished. 
 Transformers have the advantage of having no recurrent units, therefore requiring less training time than earlier recurrent neural architectures (RNNs) such as long short-term memory (LSTM).[2] Later variations have been widely adopted for training large language models (LLMs) on large (language) datasets.[3]
 
The modern version of the transformer

In [37]:
# Test PDF scraper
pdf_path = "transformer.pdf"  # Ensure this file exists in your project folder
pdf_result = research_paper_scraper.invoke(pdf_path)
print("PDF Scraper Output:", pdf_result[:200], "...")
print(pdf_result)


PDF Scraper Output: Abstract ThisdocumentprovidesanoverviewoftheTransformerarchitecture, acorner- stone of modern artificial intelligence, particularly in natural language processing and computer vision. We explore its s ...
Abstract ThisdocumentprovidesanoverviewoftheTransformerarchitecture, acorner- stone of modern artificial intelligence, particularly in natural language processing and computer vision. We explore its structure, key components, applications, and impactonAIadvancements. 1 Introduction The Transformer architecture, introduced in the seminal paper “Attention is All You Need”byVaswanietal. (2017),revolutionizedartificialintelligencebyprovidingahighly efficient and scalable model for sequence-to-sequence tasks. Unlike its predecessors, suchasrecurrentneuralnetworks(RNNs),Transformersrelyentirelyonattentionmech- anisms,enablingfastertrainingandsuperiorperformanceonlargedatasets. 2 Transformer Architecture The Transformer is built on a novel architecture that leverages self

In [46]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.vectorstores import Chroma
import shutil

# Clear existing vector store to avoid duplicates
shutil.rmtree(r"C:\Users\punithb\capstone project 2.0\chroma_db1", ignore_errors=True)

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create documents (assuming web_result and pdf_result are defined)
documents = []
if "Error" not in web_result:
    documents.append(web_result)
if "Error" not in pdf_result:
    documents.append(pdf_result)

# Split documents into chunks and assign metadata
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50, separator="\n")
chunks = []
metadatas = []
for i, doc in enumerate(documents):
    doc_chunks = text_splitter.split_text(doc)
    chunks.extend(doc_chunks)
    metadatas.extend([{"source": f"document_{i+1}", "type": "web" if i == 0 else "pdf"}] * len(doc_chunks))

# Create vector store
try:
    vectorstore = Chroma.from_texts(
        texts=chunks,
        embedding=embeddings,
        collection_name="rag_collection",
        persist_directory=r"C:\uses\punithb\capstone project 2.0\chroma_db1",
        metadatas=metadatas
    )
    print("Vector store created successfully!")
except Exception as e:
    print(f"Error creating vector store: {str(e)}")

# Perform similarity search
try:
    results = vectorstore.similarity_search("What is a transformer in NLP?", k=3)
    for i, result in enumerate(results):
        print(f"Result {i+1} (Metadata: {result.metadata}): {result.page_content[:100]}...")
except Exception as e:
    print(f"Search error: {str(e)}")

Created a chunk of size 577, which is longer than the specified 500


Vector store created successfully!
Result 1 (Metadata: {'source': 'document_1', 'type': 'web'}): Contents Transformer (deep learning architecture) In deep learning, the transformer is a neural netw...
Result 2 (Metadata: {'source': 'document_1', 'type': 'web'}): The modern version of the transformer was proposed in the 2017 paper "Attention Is All You Need" by ...
Result 3 (Metadata: {'source': 'document_2', 'type': 'pdf'}): Abstract ThisdocumentprovidesanoverviewoftheTransformerarchitecture, acorner- stone of modern artifi...


In [49]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import shutil

# Function to add space after each letter in a string (for PDF chunks)
def add_space_after_letters(text):
    return "".join(c + " " if c.isalnum() else c for c in text)

# Clear existing vector store
shutil.rmtree(r"C:\Users\punithb\capstone project 2.0\chroma_db1", ignore_errors=True)

# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Create documents (assuming web_result and pdf_result are defined)
documents = []
if "Error" not in web_result:
    documents.append(web_result)
if "Error" not in pdf_result:
    documents.append(pdf_result)

# Split documents into chunks and assign metadata
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20, separator="\n")
chunks = []
metadatas = []
for i, doc in enumerate(documents):
    doc_chunks = text_splitter.split_text(doc)
    chunks.extend(doc_chunks)
    metadatas.extend([{"source": f"document_{i+1}", "type": "web" if i == 0 else "pdf"}] * len(doc_chunks))

# Create and persist vector store
vectorstore = Chroma.from_texts(
    texts=chunks,
    embedding=embeddings,
    collection_name="rag_collection",
    persist_directory=r"C:\Users\punithb\capstone project 2.0\chroma_db1",
    metadatas=metadatas
)

# Load vector store
vectorstore = Chroma(
    collection_name="rag_collection",
    embedding_function=embeddings,
    persist_directory=r"C:\Users\punithb\capstone project 2.0\chroma_db1"
)

# Set up retriever with MMR
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 3})

# Format documents with metadata, adding spaces for PDF chunks
def format_docs(docs):
    formatted_docs = []
    for doc in docs:
        if not isinstance(doc.page_content, str):
            continue
        content = doc.page_content
        if doc.metadata.get("type") == "pdf":
            content = add_space_after_letters(content)
        formatted_docs.append(f"Source: {doc.metadata.get('source', 'unknown')} ({doc.metadata.get('type', 'unknown')})\nContent: {content}")
    return "\n\n".join(formatted_docs)

# RAG prompt
rag_prompt = ChatPromptTemplate.from_template(
    "Use the following context to answer the question: \n{context}\n\nQuestion: {question}\nAnswer:"
)

# Create RAG chain (assuming llm is defined)
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

# Test RAG pipeline
query = "What is singing?"
response = rag_chain.invoke(query)
print("RAG Answer:\n" + "-"*50 + "\n" + response + "\n" + "-"*50)
sources = retriever.invoke(query)
print("Sources:\n" + "-"*50)
for i, doc in enumerate(sources):
    print(f"Source {i+1} (Metadata: {doc.metadata}):\n{doc.page_content[:100]}...\n")

Created a chunk of size 577, which is longer than the specified 200
Created a chunk of size 311, which is longer than the specified 200


RAG Answer:
--------------------------------------------------
Unfortunately, the provided context does not mention singing. It appears to be about the transformer architecture in deep learning, its applications, and its advantages over previous architectures. Therefore, I cannot provide a relevant answer to the question about singing.
--------------------------------------------------
Sources:
--------------------------------------------------
Source 1 (Metadata: {'source': 'document_1', 'type': 'web'}):
Contents Transformer (deep learning architecture) In deep learning, the transformer is a neural netw...

Source 2 (Metadata: {'source': 'document_1', 'type': 'web'}):
The modern version of the transformer was proposed in the 2017 paper "Attention Is All You Need" by ...

Source 3 (Metadata: {'source': 'document_1', 'type': 'web'}):
Transformers have the advantage of having no recurrent units, therefore requiring less training time...

